<h1>m2w1_JSON_weather_watkins_challenge_1122</h2>

<b>Challenge Steps</b>
<ol>
<li>New Project - Done</li>
<li>New Venv - Done</li>
<li>New Jupyter - Done</li>
<li>Download JSON file "nws_weather_4HourForecast.json" to the root directory of your python project.</li>
<li>Import the json file into a Python dictionary.</li>
<li>Iterate through the "Properties" key, loading keys and values into a Pandas dataframe</li>
<li>Display as a table the following columns:
"startTime" ,"shortForecast" ,"temperature" ,"probabilityOfPrecipitation" ,"relativeHumidity"
</li>
</ol>
 
<!--

<b>More Challenge<\b>
<ol>
The above not enough for you? Try these:
- download the JSON file directly into your Jupyter notebook by making a URL request to the address shown above
- rename columns to shorter, easier-to-display names
- reformat the datetime value to something easier to read for someone in its offset timezone (US Eastern)
- convert the dewpoint's Celsius value to Fahrenheit
- concatenate columns "temperature" and "temperatureUnit" so that the reader better understands the "temperature" value
- use a json datafile that you download directly from the source "https://api.weather.gov"
</ol>
-->


In [3]:
import pandas as pd

In [5]:
import os
print(os.getcwd())

#weather_file_1 = pd.read_json('nws_weather_4HourForecast.json')

d:\00 Development\Code KY\challenges
